# Benchmark Manifold GP Supervised Learning

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from time import time
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.models.vanilla_gp import VanillaGP
from gpytorch.priors import NormalPrior, GammaPrior

## Dataset Preprocessing

### Load & Settings

In [2]:
# bike, buzz_tomshardware, buzz_twitter, ctslices, elevators, protein, song, mnist, mnist_single
dataset = 'ctslices'
samples_split = 0.75
preprocess = True
normalize_features = False
normalize_labels = True
# torch.manual_seed(1337)
sample_dist = False
train_dist = False

data = np.load('../datasets/'+dataset+'.npy')
data = data[np.random.permutation(data.shape[0])]
num_samples = int(samples_split * len(data))
sampled_x, sampled_y = data[:num_samples, :-1], data[:num_samples, -1]
test_x, test_y = data[num_samples:, :-1], data[num_samples:,-1]

# x = x - x.min(0)[0]
# x = 2 * (x/ x.max(0)[0]) - 1
# mu, std = x.mean(axis=0), x.std(axis=0)
# x = (x - mu)/(std+ 1e-6)

In [3]:
if preprocess:
    # remove coincident points
    sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    sampled_y = sampled_y[id_unique]

    # # cut between x% and y% percentile of distances
    # num_avg = 1
    # p_start, p_end = 0.1, 0.9
    # num_samples = sampled_x.shape[0]
    # import faiss
    # res = faiss.StandardGpuResources()
    # knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    # knn.train(sampled_x)
    # knn.add(sampled_x)
    # v = np.sqrt(knn.search(sampled_x, num_avg+1)[0][:,1:])
    # idx = np.argsort(v.mean(axis=1))
    # idx = np.delete(idx, np.arange(int(num_samples*p_start),int(num_samples*p_end)))
    # sampled_x = np.delete(sampled_x, idx, axis=0)
    # sampled_y = np.delete(sampled_y, idx)
    # del knn

    # randomized dataset
    idx = np.random.permutation(sampled_x.shape[0])
    sampled_x = sampled_x[idx]
    sampled_y = sampled_y[idx]
m = sampled_x.shape[0]

In [4]:
if sample_dist:
    import faiss
    import matplotlib.pyplot as plt
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(sampled_x)
    knn.add(sampled_x)
    v, i = knn.search(sampled_x, 2)
    v = np.sqrt(v[:,1:]).mean(axis=1)
    
    plt.hist(np.sort(v), density=False, bins=100)  # density=False would make counts
    plt.ylabel('Probability')
    plt.xlabel('Data')
    del knn

### Trainset & Testset

In [5]:
train_split = int(0.05* m)
train_x, train_y = sampled_x[:train_split], sampled_y[:train_split]

train_x, train_y = torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()
test_x, test_y = torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

In [6]:
if train_dist:
    import faiss
    import matplotlib.pyplot as plt
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, train_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(train_x)
    knn.add(train_x)
    v, i = knn.search(test_x, 2)
    v = np.sqrt(v[:,1:]).mean(axis=1)
    
    plt.hist(np.sort(v), density=False, bins=100)  # density=False would make counts
    plt.ylabel('Probability')
    plt.xlabel('Data')
    del knn

### Move Data to Device

In [7]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Vanilla Model

In [8]:
%%capture
model_vanilla = VanillaGP(
    train_x, 
    train_y, 
    gpytorch.likelihoods.GaussianLikelihood(), 
    gpytorch.kernels.ScaleKernel(
        # gpytorch.kernels.RBFKernel()
        gpytorch.kernels.MaternKernel(nu=2.5)
    ) # gpytorch.kernels.RBFKernel(), gpytorch.kernels.RFFKernel(100)
).to(device)
model_vanilla_name = dataset + '_vanilla'

In [9]:
model_vanilla.vanilla_train(lr=1e-1, iter=200, verbose=True)

Iteration: 0, Loss: 1.423, Noise Variance: 0.833, Signal Variance: 0.833, Lengthscale: 0.693
Iteration: 1, Loss: 1.411, Noise Variance: 0.803, Signal Variance: 0.803, Lengthscale: 0.744
Iteration: 2, Loss: 1.398, Noise Variance: 0.774, Signal Variance: 0.774, Lengthscale: 0.798
Iteration: 3, Loss: 1.388, Noise Variance: 0.745, Signal Variance: 0.746, Lengthscale: 0.855
Iteration: 4, Loss: 1.379, Noise Variance: 0.718, Signal Variance: 0.720, Lengthscale: 0.913
Iteration: 5, Loss: 1.370, Noise Variance: 0.691, Signal Variance: 0.697, Lengthscale: 0.975
Iteration: 6, Loss: 1.361, Noise Variance: 0.667, Signal Variance: 0.678, Lengthscale: 1.038
Iteration: 7, Loss: 1.354, Noise Variance: 0.643, Signal Variance: 0.666, Lengthscale: 1.105
Iteration: 8, Loss: 1.343, Noise Variance: 0.622, Signal Variance: 0.661, Lengthscale: 1.174
Iteration: 9, Loss: 1.329, Noise Variance: 0.602, Signal Variance: 0.663, Lengthscale: 1.245
Iteration: 10, Loss: 1.313, Noise Variance: 0.582, Signal Variance: 0.

In [10]:
# torch.save(model_vanilla.state_dict(), '../outputs/models/' + model_vanilla_name + '.pth')
# model_vanilla.load_state_dict(torch.load('../outputs/models/' + model_vanilla_name + '.pth'))

## Geometric Model

In [11]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=3, # 6
        nodes=train_x,
        neighbors=10, # 100
        operator="randomwalk",
        method="exact",
        modes=1000, # 500
        ball_scale=10.0,
        ball_decay=0.01,
        prior_bandwidth=False,
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel).to(device)
model_name = dataset+'_sup_nu'+str(int(kernel.base_kernel.nu.item()))+'_k'+str(kernel.base_kernel.neighbors)

## Train

In [12]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.epsilon': 1.5, # kernel.base_kernel.epsilon_prior.sample()
    'covar_module.base_kernel.lengthscale': 10.0, # 10
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

In [13]:
model.manifold_informed_train(lr=1e-1, iter=200, 
                              decay_step_size=1000, decay_magnitude=1.0, 
                              norm_step_size=10, norm_rand_vec=100, 
                              verbose=True, save=False)
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())
torch.cuda.empty_cache()

Iter: 0, Loss: -1409.684, NoiseVar: 0.010, SignalVar: 0.00104, Lengthscale: 10.000, Epsilon: 1.500
Iter: 1, Loss: -4755.744, NoiseVar: 0.009, SignalVar: 0.00094, Lengthscale: 9.900, Epsilon: 1.423
Iter: 2, Loss: -8029.049, NoiseVar: 0.008, SignalVar: 0.00085, Lengthscale: 9.800, Epsilon: 1.348
Iter: 3, Loss: -11524.930, NoiseVar: 0.007, SignalVar: 0.00077, Lengthscale: 9.702, Epsilon: 1.275
Iter: 4, Loss: -15890.256, NoiseVar: 0.007, SignalVar: 0.00070, Lengthscale: 9.604, Epsilon: 1.204
Iter: 5, Loss: -22603.377, NoiseVar: 0.007, SignalVar: 0.00063, Lengthscale: 9.507, Epsilon: 1.134
Iter: 6, Loss: -32528.697, NoiseVar: 0.007, SignalVar: 0.00057, Lengthscale: 9.412, Epsilon: 1.068
Iter: 7, Loss: -44279.773, NoiseVar: 0.008, SignalVar: 0.00051, Lengthscale: 9.321, Epsilon: 1.004
Iter: 8, Loss: -55947.250, NoiseVar: 0.008, SignalVar: 0.00047, Lengthscale: 9.237, Epsilon: 0.942
Iter: 9, Loss: -61324.812, NoiseVar: 0.009, SignalVar: 0.00042, Lengthscale: 9.173, Epsilon: 0.881
Iter: 10, Lo

In [14]:
# torch.save(model.state_dict(), '../outputs/models/'+model_name+'.pth')
# model.load_state_dict(torch.load('../outputs/models/'+model_name+'.pth'))

In [15]:
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())

NoiseVar:  0.032320536673069 SignalVar:  0.36080968379974365 Lengthscale:  13.694597244262695 Epsilon 1.0928875207901


### Extract EigenPairs

In [16]:
# kernel.base_kernel.method = 'exact'
# kernel.base_kernel.modes = 2000
kernel.base_kernel.generate_eigenpairs()
evals, evecs = kernel.base_kernel.eigenvalues.cpu(), kernel.base_kernel.eigenvectors.cpu()

exact


### Train with Fixed number of Eigenfunctions

In [17]:
model.vanilla_train(lr=1e-1, iter=100, verbose=True)
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())

Iter: 0, Loss: -0.523, NoiseVar: 0.032, SignalVar: 0.36081, Lengthscale: 13.695, Epsilon: 1.093
Iter: 1, Loss: -0.561, NoiseVar: 0.029, SignalVar: 0.39217, Lengthscale: 13.795, Epsilon: 1.093
Iter: 2, Loss: -0.598, NoiseVar: 0.027, SignalVar: 0.42529, Lengthscale: 13.894, Epsilon: 1.093
Iter: 3, Loss: -0.633, NoiseVar: 0.024, SignalVar: 0.45981, Lengthscale: 13.994, Epsilon: 1.093
Iter: 4, Loss: -0.667, NoiseVar: 0.022, SignalVar: 0.49519, Lengthscale: 14.093, Epsilon: 1.093
Iter: 5, Loss: -0.699, NoiseVar: 0.020, SignalVar: 0.53077, Lengthscale: 14.191, Epsilon: 1.093
Iter: 6, Loss: -0.728, NoiseVar: 0.018, SignalVar: 0.56579, Lengthscale: 14.289, Epsilon: 1.093
Iter: 7, Loss: -0.756, NoiseVar: 0.016, SignalVar: 0.59943, Lengthscale: 14.385, Epsilon: 1.093
Iter: 8, Loss: -0.782, NoiseVar: 0.015, SignalVar: 0.63089, Lengthscale: 14.482, Epsilon: 1.093
Iter: 9, Loss: -0.806, NoiseVar: 0.013, SignalVar: 0.65942, Lengthscale: 14.577, Epsilon: 1.093
Iter: 10, Loss: -0.827, NoiseVar: 0.012,

In [18]:
print("NoiseVar: ", likelihood.noise.item(), "SignalVar: ", kernel.outputscale.item(), 
      "Lengthscale: ", kernel.base_kernel.lengthscale.item(), "Epsilon", kernel.base_kernel.epsilon.item())

NoiseVar:  0.006770143285393715 SignalVar:  0.47343960404396057 Lengthscale:  23.675678253173828 Epsilon 1.0928875207901


In [19]:
# torch.save(model.state_dict(), '../outputs/models/'+model_name+'.pth')
# model.load_state_dict(torch.load('../outputs/models/'+model_name+'.pth'))

## Evaluation

In [20]:
%%capture
model_vanilla.likelihood.eval()
model_vanilla.eval()
likelihood.eval()
model.eval()
torch.cuda.empty_cache()
model.vanilla_model = model_vanilla

In [21]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    model.posterior(test_x, noise=True)
    bump_scale = 1-kernel.base_kernel.bump_function(test_x)

/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:337: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.026487696915864944 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


### Vanilla

In [22]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("vanilla")
    covar = model.posterior_vanilla.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
print("RMSE: ", rmse)
print("NLL: ", nll)
model._clear_cache()
model_vanilla._clear_cache()
torch.cuda.empty_cache()

RMSE:  tensor(0.0584, device='cuda:0')
NLL:  tensor(-1.3972, device='cuda:0')


### Manifold

In [23]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("manifold")
    covar = model.posterior_manifold.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
print("RMSE: ", rmse)
print("NLL: ", nll)
model._clear_cache()
model_vanilla._clear_cache()
torch.cuda.empty_cache()

RMSE:  tensor(0.0977, device='cuda:0')
NLL:  tensor(-0.7909, device='cuda:0')


### Hybrid

In [24]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    error = test_y - model.mean("hybrid")
    covar = model.posterior_manifold.lazy_covariance_matrix.evaluate_kernel() 
    + torch.outer(bump_scale,bump_scale) * model.posterior_vanilla.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    rmse = (error.square().mean()).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/error.size(-1)
print("RMSE: ", rmse)
print("NLL: ", nll)
model._clear_cache()
model_vanilla._clear_cache()
torch.cuda.empty_cache()

RMSE:  tensor(0.0977, device='cuda:0')
NLL:  tensor(-0.7865, device='cuda:0')
